In [1]:
!pip install -q -U transformers  peft datasets tqdm torch accelerate evaluate deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 

In [2]:
from transformers import AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch
from datasets import load_dataset
import os
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset



In [3]:
from datasets import load_dataset

def preprocess_dataset(dataset_name):
    # Load a dataset
    dataset = load_dataset(dataset_name)

    # Function to preprocess an example based on detected feature types
    def preprocess_function(example):
        new_example = {}
        # Loop over all features in the example
        for feature_name, feature_value in example.items():
            if isinstance(feature_value, str):  # For text features
                # Apply text preprocessing like tokenization, lowercasing, etc.
                new_example[feature_name] = feature_value.lower()  # Example operation
            elif isinstance(feature_value, list):  # For categorical features
                if isinstance(feature_value[0], int):  # Label encoded list
                    # Convert numeric labels to strings if names are provided in the feature
                    label_feature = dataset["train"].features[feature_name]
                    if hasattr(label_feature, 'names'):
                        new_example[feature_name] = [label_feature.names[label] for label in feature_value]
                    else:
                        new_example[feature_name] = feature_value  # Unchanged
                else:
                    # Handle other list data types if necessary
                    pass
            else:  # For numerical or other types of features
                # Apply numerical preprocessing like normalization, scaling, etc.
                new_example[feature_name] = feature_value  # Example operation

        return new_example

    # Apply the preprocessing function to all splits in the dataset
    processed_dataset = dataset.map(preprocess_function, batched=True, num_proc=1)

    return processed_dataset

# Replace 'some_specific_name' with the actual name of the dataset you want to load
dataset_name = "LDJnr/Capybara"
processed_dataset = preprocess_dataset(dataset_name)

# Display the processed dataset and the first example of the train split
print(processed_dataset)
print(len(processed_dataset["train"]))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16006 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'conversation'],
        num_rows: 16006
    })
})
16006


In [ ]:
processed_dataset["train"][0]

{'source': 'General-Instruct',
 'conversation': [{'input': 'Analyze and compare the two classical music composers Wolfgang Amadeus Mozart and Ludwig van Beethoven on aspects such as musical style, philosophical beliefs, and overall influence on music history.\n',
   'output': "Wolfgang Amadeus Mozart (1756–1791) and Ludwig van Beethoven (1770–1827) were two titans of classical music history who, though distinct in their musical styles, left an indelible impact on music. Mozart, known for his melodic genius, created an extensive repertoire covering symphonies, operas, chamber works, and choral music. His music exhibits the Enlightenment era's ideals and prioritizes balance, beauty, and lyrical expressions.\n\nBeethoven's music reflects a dramatic shift, characterized by driving rhythms, expressive harmonies, and passionate intensity. His work often represents Romanticism's essence, placing more emphasis on individualism and emotion. As a bridge between the classical and romantic eras, B

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name_or_path = "gpt2"
tokenizer_name_or_path = "gpt2"
dataset_name = "twitter_complaints"
text_column = 'source'
label_column = 'conversation'
max_length = 64
lr = 1e-3
num_epochs = 50
batch_size = 8

In [6]:
processed_dataset['train'].features.items()

dict_items([('source', Value(dtype='string', id=None)), ('conversation', [{'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None)}])])

In [7]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained('gpt2')
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def preprocess_dataset(dataset_name):
    # Load a dataset
    dataset = load_dataset(dataset_name)

    # Function to preprocess an example based on detected feature types
    def preprocess_function(example):
        new_example = {}
        # Loop over all features in the example
        for feature_name, feature_value in example.items():
            if isinstance(feature_value, str):  # For text features
                # Apply text preprocessing like tokenization, lowercasing, etc.
                new_example[feature_name] = feature_value.lower()  # Example operation
            elif isinstance(feature_value, list):  # For categorical features
                if isinstance(feature_value[0], int):  # Label encoded list
                    # Convert numeric labels to strings if names are provided in the feature
                    label_feature = dataset["train"].features[feature_name]
                    if hasattr(label_feature, 'names'):
                        new_example[feature_name] = [label_feature.names[label] for label in feature_value]
                    else:
                        new_example[feature_name] = feature_value  # Unchanged
                else:
                    # Handle other list data types if necessary
                    pass
            else:  # For numerical or other types of features
                # Apply numerical preprocessing like normalization, scaling, etc.
                new_example[feature_name] = feature_value  # Example operation

        return new_example

    # Apply the preprocessing function to all splits in the dataset
    processed_dataset = dataset.map(preprocess_function, batched=True, num_proc=1)

    return processed_dataset



def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=max_length, truncation=True, padding='max_length', add_special_tokens=False)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]

    # model_inputs["labels"] = [
    #     [-100 if input_id == tokenizer.pad_token_id else label_id for input_id, label_id in zip(input_seq, label_seq)]
    #     for input_seq, label_seq in zip(model_inputs["input_ids"], labels["input_ids"])
    # ]

    return model_inputs


# Load and preprocess the dataset
dataset_name = "HuggingFaceH4/ultrafeedback_binarized"
processed_dataset = preprocess_dataset(dataset_name)

dataset=load_dataset("HuggingFaceH4/ultrafeedback_binarized")

# Define your columns for text and labels based on your dataset
text_column =dataset['test_sft'].column_names[0] # Replace with actual text column name
label_column = dataset['test_sft'].column_names[1]  # Replace with actual label column name

# Additional required parameters
max_length = 512  # Example max length, adjust as needed

# Now apply your tokenizer processing function to the processed_dataset
processed_datasets = processed_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=processed_dataset["test_sft"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

#Use the processed_datasets for training
train_dataset = processed_datasets["test_sft"][0]

In [ ]:
print(len(train_dataset['input_ids'][:]))

512


In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import torch
from datasets import load_dataset
import os
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset




device = "cuda"
model_name_or_path = "bigscience/bloomz-7b1"
tokenizer_name_or_path = "bigscience/bloomz-7b1"
dataset_name = "twitter_complaints"
text_column = "Tweet text"
label_column = "text_label"
max_length = 64
lr = 1e-3
num_epochs = 50
batch_size = 8





In [ ]:
# data preprocessing
from datasets import load_dataset


dataset_name=''
dataset = load_dataset( dataset_name)


classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
print(classes)
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
print(dataset)
dataset["train"][0]

In [ ]:
from datasets import load_dataset
def function(value):
    # Your function here. This is an identity function as an example.
    return value
# Load the dataset
dataset = load_dataset("Tele-AI/TeleChat-PTD")
for  split in dataset.keys():
   print(split)
for split_key in dataset.keys():

 # Access the split
  data_split = dataset[split_key]
  print(data_split)

  # Manipulate keys and values
  for column_key in data_split.column_names:

    # Apply the function to the column
    updated_column = data_split.map(
        lambda example: {column_key: function(example[column_key])},
        remove_columns=[column_key]
    )
    print(updated_column)
    # Update the column in the split
    data_split = updated_column
  # Update the split in the original dataset
  dataset[split_key] = data_split

Resolving data files:   0%|          | 0/189 [00:00<?, ?it/s]

OSError: [Errno 28] No space left on device

In [ ]:
print(dataset.keys())

In [ ]:
dataset = load_dataset('fka/awesome-chatgpt-prompts')

# Get the list of string columns in the dataset
string_columns = [k for k, v in dataset['train'].features.items() if isinstance(v, datasets.Value) and v.dtype == 'string']

# Check if each column exists in the dataset before sorting
sorted_columns = sorted([col for col in string_columns if col in dataset['train'].column_names],
                        key=lambda col: max(len(x) for x in dataset['train'][col]),
                        reverse=True)

print(sorted_columns)

In [ ]:
from datasets import load_dataset
def function(value):
    # Your function here. This is an identity function as an example.
    return value
# Load the dataset
dataset = load_dataset("fka/awesome-chatgpt-prompts")

for split_key in dataset.keys():

 # Access the split
  data_split = dataset[split_key]
  print(data_split)

  # Manipulate keys and values
  for column_key in data_split.column_names:

    # Apply the function to the column
    updated_column = data_split.map(
        lambda example: {column_key: function(example[column_key])},
        remove_columns=[column_key]
    )
    print(updated_column)
    # Update the column in the split
    data_split = updated_column
  # Update the split in the original dataset
  dataset[split_key] = data_split

Dataset({
    features: ['act', 'prompt'],
    num_rows: 153
})
Dataset({
    features: ['act', 'prompt'],
    num_rows: 153
})


Map:   0%|          | 0/153 [00:00<?, ? examples/s]

Dataset({
    features: ['act', 'prompt'],
    num_rows: 153
})


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import datasets
from transformers import AutoTokenizer
# Initialize the tokenizer for a specific model
tokenizer = AutoTokenizer.from_pretrained('gpt2')
# If the tokenizer does not have a pad token, set it to the eos token
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def detect_text_and_label_columns(dataset):
    """
    This function attempts to automatically detect the text and label columns.
    The heuristic used here is to find the two longest string columns.
    """
    # Filter out columns with string data type
    string_columns = [k for k, v in dataset.features.items() if isinstance(v, datasets.Value) and v.dtype == 'string']
    # Sort string columns by their maximum length
    sorted_columns = sorted(string_columns, key=lambda col: max(len(x) for x in dataset[col]), reverse=True)

    if len(sorted_columns) < 2:
        raise ValueError("Unable to automatically detect text and label columns. Please specify them manually.")

    # Assume text column has the longest strings, and label column has the second-longest strings
    return sorted_columns[0], sorted_columns[1]

def preprocess_function(examples, text_column, label_column, max_length=512):
    """
    Tokenize the text and labels from the dataset.
    """
    # Prepare inputs and targets for the tokenizer
    inputs = [f"{text_column}: {x} Label:" for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]

    # Tokenize the inputs and targets
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=max_length, truncation=True, padding='max_length', add_special_tokens=False)

    # Replace padding token id in labels with -100 so it's ignored by the loss function
    model_inputs["labels"] = [
        [-100 if input_id == tokenizer.pad_token_id else label_id for input_id, label_id in zip(input_seq, label_seq)]
        for input_seq, label_seq in zip(model_inputs["input_ids"], labels["input_ids"])
    ]

    return model_inputs

def preprocess_dataset(dataset_name):
    """
    Load and preprocess the dataset.
    """
    # Load the dataset
    dataset = load_dataset(dataset_name)

    # Automatically detect text and label columns
    text_column, label_column = detect_text_and_label_columns(dataset["train"])

    # Define maximum token length for the model inputs
    max_length = 512

    # Apply the preprocess_function to all splits in the dataset
    processed_datasets = dataset.map(
        lambda examples: preprocess_function(examples, text_column, label_column, max_length),
        batched=True,
        num_proc=1,
        remove_columns=dataset["train"].column_names,
        load_from_cache_file=False,
        desc="Running tokenizer on dataset",
    )

    return processed_datasets, text_column, label_column

# Specify the dataset name
dataset_name = "ldjnr/capybara"

# Preprocess the dataset
processed_datasets, text_column, label_column = preprocess_dataset(dataset_name)

# The processed_datasets can now be used for training a model
train_dataset = processed_datasets["train"]

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("KBlueLeaf/danbooru2023-sqlite")

# Define a function to manipulate keys and values
def manipulate_dataset(dataset, function):
    for split_key in dataset.keys():
        # Access the split
        data_split = dataset[split_key]

        # Manipulate keys and values
        for column_key in data_split.column_names:
            # Apply the function to the column
            updated_column = data_split.map(
                lambda example: {column_key: function(example[column_key])},
                remove_columns=[column_key]
            )

            # Update the column in the split
            data_split = updated_column

        # Update the split in the original dataset
        dataset[split_key] = data_split

    return dataset

# Define a function to apply
def function(value):
    # Your function here. This is an identity function as an example.
    return value

# Apply the function to the dataset
dataset = manipulate_dataset(dataset, function)


In [ ]:
dataset

In [ ]:
train_dataset = processed_datasets["train"]
print('len:',len(train_dataset ))
print(train_dataset[0])


len: 16006
{'input_ids': [10459, 1058, 3611, 12, 43993, 36052, 1058, 220, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)
print(processed_datasets)

train_dataset = processed_datasets["train"]


In [ ]:

classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
print(classes)
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
print(dataset)
dataset["train"][0]

In [ ]:
from datasets import load_dataset
dataset = load_dataset('LDJnr/Capybara')
print(print(dataset['train'].column_names)
)

['source', 'conversation']
None


In [ ]:
print(dataset["train"])




In [ ]:
label_feature = dataset["train"].features['source']


Value(dtype='string', id=None)


In [ ]:
def extract_features(dataset, split:str):
    feature_names = dataset[split].features.keys()
    return list(feature_names)
def preprocess_function(example):
    new_example = {}
    # Loop over all features in the example
    for feature_name, feature_value in example.items():
        if isinstance(feature_value, str):  # For text features
            # Apply text preprocessing like tokenization, lowercasing, etc.
            new_example[feature_name] = feature_value.lower()  # Example operation
        elif isinstance(feature_value, list):  # For categorical features
            if isinstance(feature_value[0], int):  # Label encoded list
                # Convert numeric labels to strings if names are provided in the feature
                label_feature = dataset["train"].features[feature_name]
                if hasattr(label_feature, 'names'):
                    new_example[feature_name] = [label_feature.names[label] for label in feature_value]
                else:
                    new_example[feature_name] = feature_value  # Unchanged
            else:
                # Handle other list data types if necessary
                pass
        else:  # For numerical or other types of features
            # Apply numerical preprocessing like normalization, scaling, etc.
            new_example[feature_name] = feature_value  # Example operation

    return new_example




In [ ]:
preprocess_function(dataset['train'].features)

{'source': Value(dtype='string', id=None)}

In [ ]:
x=dataset['train'].features.values()

In [ ]:
x

dict_values([Value(dtype='string', id=None), [{'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None)}]])

In [ ]:
for x in

In [ ]:
x=extract_features(dataset,'train')


dataset = dataset.map(
    lambda x: {"source": [classes[label] for label in extract_features(dataset,'train')]},
    batched=True,
    num_proc=1,
)
for y in x:
  print(y)

In [ ]:
from datasets import load_dataset, DatasetDict

# Load a dataset (replace 'your_dataset' with the specific dataset you're working with)
dataset = load_dataset('your_dataset')

# Function to extract features from the dataset
def extract_features(dataset):
    feature_names = dataset.features.keys()
    return list(feature_names)

# Define a preprocessing function that accepts the entire example and the feature names
def preprocess_function(example, feature_names):
    # Placeholder for preprocessed example
    preprocessed_example = {}

    for feature_name in feature_names:
        feature_value = example[feature_name]

        # Apply a different preprocessing step depending on the feature type
        # Here, you can check the type of feature and apply suitable preprocessing
        # e.g., tokenization for text, normalization for numerical data, etc.

        # Example of text preprocessing
        if isinstance(feature_value, str):  # assuming the feature is text
            # Apply some text preprocessing, e.g., lowercasing
            preprocessed_example[feature_name] = feature_value.lower()

        # Example of numerical data preprocessing
        elif isinstance(feature_value, (int, float)):  # assuming the feature is numerical
            # Apply numerical preprocessing, e.g., standardization
            pass

        # Example of categorical data preprocessing
        elif isinstance(feature_value, list):  # assuming the feature is categorical
            # Apply categorical preprocessing, e.g., label encoding
            pass

    return preprocessed_example

# First, let's extract the column names
feature_names = extract_features(dataset['train'])

# Process the dataset by applying the preprocessing function
# to all splits in the dataset
processed_dataset = DatasetDict({
    split: dataset[split].map(
        lambda x: preprocess_function(x, feature_names),
        batched=True,
        num_proc=1
    ) for split in dataset.keys()
})

# Check a part of the processed dataset
print(processed_dataset)
print(processed_dataset['train'][0:10])

In [ ]:

dataset = dataset.map(
    lambda x: {"source": [classes[label] for label in x["conversation"]]},
    batched=True,
    num_proc=1,
)
print(dataset)
dataset["train"][0:10]

In [12]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
# print(target_max_length)


In [13]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [14]:
processed_datasets = processed_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=processed_dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)
print(processed_datasets)

train_dataset = processed_datasets["train"]




Running tokenizer on dataset:   0%|          | 0/16006 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1207 > 1024). Running this sequence through the model will result in indexing errors


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16006
    })
})


In [15]:
print(train_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 16006
})


In [16]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    return model_inputs

In [19]:

processed_datasets = processed_dataset.map(
    test_preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=processed_dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/16006 [00:00<?, ? examples/s]

In [21]:
eval_dataset = processed_datasets["train"]
print("eval_dataset:",eval_dataset)
test_dataset = processed_datasets["train"]
print("test_dataset:",test_dataset)

eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
print(next(iter(eval_dataloader)))
print(next(iter(test_dataloader)))

eval_dataset: Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 16006
})
test_dataset: Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 16006
})
{'input_ids': tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 10459,  1058,  3611,    12,
         43993, 36052,  1058,   220],
        [50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256

In [22]:
from peft import PeftModel, PeftConfig

max_memory = { "cpu": "30GB"}
peft_model_id = "latent-consistency/lcm-lora-sdv1-5"
base_model_name_or_path='gpt2'
# config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(base_model_name_or_path, device_map="auto", max_memory=max_memory)
# model = PeftModel.from_pretrained(model, base_model_name_or_path, device_map="auto", max_memory=max_memory)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [23]:
model.hf_device_map

OrderedDict([('', 'cpu')])

In [28]:
processed_datasets["train"][0]

{'input_ids': [50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  10459,
  1058,
  3611,
  12,
  43993,
  36052,
  1058,
  220],
 'attention_mask': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [29]:
model.eval()
i = 89
text_column = 'source'
label_column = 'conversation'
inputs = tokenizer(f'{text_column} : {processed_datasets["train"][i]["input_ids"]} Label : ', return_tensors="pt")
# print(processed_datasets["train"][i]["attention_mask"])
# print(inputs)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[10459,  1058,   685,  1120, 11645,    11,  2026, 11645,    11,  2026,
         11645,    11,  2026, 11645,    11,  2026, 11645,    11,  2026, 11645,
            11,  2026, 11645,    11,  2026, 11645,    11,  2026, 11645,    11,
          2026, 11645,    11,  2026, 11645,    11,  2026, 11645,    11,  2026,
         11645,    11,  2026, 11645,    11,  2026, 11645,    11,  2026, 11645,
            11,  2026, 11645,    11,  2026, 11645,    11,  2026, 11645,    11,
          2026, 11645,    11,  2026, 11645,    11,  2026, 11645,    11,  2026,
         11645,    11,  2026, 11645,    11,  2026, 11645,    11,  2026, 11645,
            11,  2026, 11645,    11,  2026, 11645,    11,  2026, 11645,    11,
          2026, 11645,    11,  2026, 11645,    11,  2026, 11645,    11,  2026,
         11645,    11,  2026, 11645,    11,  2026, 11645,    11,  2026, 11645,
            11,  2026, 11645,    11,  2026, 11645,    11,  2026, 11645,    11,
          2026, 11645,    11,  2026, 11645,    11,  

In [ ]:
model.eval()
eval_preds = []
for _, batch in enumerate(tqdm(eval_dataloader)):
    batch = {k: v for k, v in batch.items() if k != "labels"}
    with torch.no_grad():
        outputs = model.generate(**batch, max_new_tokens=10)
    preds = outputs[:, max_length:].detach().cpu().numpy()
    eval_preds.extend(tokenizer.batch_decode(preds, skip_special_tokens=True))

 15%|█▌        | 303/2001 [18:09<1:46:41,  3.77s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:



from datasets import load_dataset

# Load the dataset
dataset = load_dataset("LDJnr/Capybara")

# Define a function to manipulate keys and values
def manipulate_dataset(dataset, function):
    for split_key in dataset.keys():
        # Access the split
        data_split = dataset[split_key]

        # Manipulate keys and values
        for column_key in data_split.column_names:
            # Apply the function to the column
            updated_column = data_split.map(
                lambda example: {column_key: function(example[column_key])},
                remove_columns=[column_key]
            )

            # Update the column in the split
            data_split = updated_column

        # Update the split in the original dataset
        dataset[split_key] = data_split

    return dataset

# Define a function to apply
def function(value):
    # Your function here. This is an identity function as an example.
    return value

# Apply the function to the dataset
dataset = manipulate_dataset(dataset, function)


# data preprocessing
tokenizer = AutoTokenizer.from_pretrained('gpt2')
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

def detect_text_and_label_columns(dataset,split:str):
    """
    This function attempts to automatically detect the text and label columns.
    The heuristic used here is to find the two longest string columns.
    """
    # Get the list of string columns in the dataset
    string_columns = [k for k, v in dataset['train'].features.items() if isinstance(v, datasets.Value) and v.dtype == 'string']

# Check if each column exists in the dataset before sorting
    sorted_columns = sorted([col for col in string_columns if col in dataset['train'].column_names],
                        key=lambda col: max(len(x) for x in dataset['train'][col]),
                        reverse=True)

    # if len(sorted_columns) < 2:
    #     raise ValueError("Unable to automatically detect text and label columns. Please specify them manually.")
    return  sorted_columns


def preprocess_dataset(dataset_name):
    # Load a dataset
    dataset = load_dataset(dataset_name)

    # Function to preprocess an example based on detected feature types
    def preprocess_function(example):
        new_example = {}
        # Loop over all features in the example
        for feature_name, feature_value in example.items():
            if isinstance(feature_value, str):  # For text features
                # Apply text preprocessing like tokenization, lowercasing, etc.
                new_example[feature_name] = feature_value.lower()  # Example operation
            elif isinstance(feature_value, list):  # For categorical features
                if isinstance(feature_value[0], int):  # Label encoded list
                    # Convert numeric labels to strings if names are provided in the feature
                    label_feature = dataset["train"].features[feature_name]
                    if hasattr(label_feature, 'names'):
                        new_example[feature_name] = [label_feature.names[label] for label in feature_value]
                    else:
                        new_example[feature_name] = feature_value  # Unchanged
                else:
                    # Handle other list data types if necessary
                    pass
            else:  # For numerical or other types of features
                # Apply numerical preprocessing like normalization, scaling, etc.
                new_example[feature_name] = feature_value  # Example operation

        return new_example

    # Apply the preprocessing function to all splits in the dataset
    processed_dataset = dataset.map(preprocess_function, batched=True, num_proc=1)

    return processed_dataset



def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=max_length, truncation=True, padding='max_length', add_special_tokens=False)

    model_inputs["labels"] = [
        [-100 if input_id == tokenizer.pad_token_id else label_id for input_id, label_id in zip(input_seq, label_seq)]
        for input_seq, label_seq in zip(model_inputs["input_ids"], labels["input_ids"])
    ]

    return model_inputs

# Load and preprocess the dataset
dataset_name = "LDJnr/Capybara"
processed_dataset = preprocess_dataset(dataset_name)

dataset=load_dataset("LDJnr/Capybara")
sorted_columns=detect_text_and_label_columns(dataset,'train')
len(sorted_columns)
if len(sorted_columns)==1:
    text_column =sorted_columns[0]
else:
# Define your columns for text and labels based on your dataset
  text_column =sorted_columns[0] # Replace with actual text column name
  label_column = sorted_columns[1]  # Replace with actual label column name

print(text_column )


# Additional required parameters
max_length = 512  # Example max length, adjust as needed

# Now apply your tokenizer processing function to the processed_dataset
processed_datasets = processed_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=processed_dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

# Use the processed_datasets for training
train_dataset = processed_datasets["train"][0]
print(train_dataset['input_ids'])

Map:   0%|          | 0/16006 [00:00<?, ? examples/s]

Map:   0%|          | 0/16006 [00:00<?, ? examples/s]

NameError: name 'datasets' is not defined

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import multiprocessing

# Load the dataset
dataset = load_dataset("fka/awesome-chatgpt-prompts")

# Define the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Extract the split name
split_name = list(dataset.keys())[0]

# Get the list of columns
columns = dataset[split_name].column_names

# Define the advanced preprocessing function
def advanced_preprocess_function(text):
    # Implement advanced text cleaning and normalization
    # For example, lowercasing, removing special characters, etc.
    # This is a placeholder for your actual preprocessing logic
    processed_text = text.lower()
    # Add more preprocessing steps as needed
    return processed_text

# Define the advanced tokenization function
def advanced_tokenize_function(examples):
    # Tokenize with dynamic batching and special token handling
    # Note: We pass the entire batch of examples to the tokenizer
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

# Define the advanced loader function for preprocessing
def advanced_preprocess_dataset(dataset, preprocess_function):
    for split in dataset.keys():
        dataset[split] = dataset[split].map(
            lambda examples: {column: preprocess_function(examples[column]) for column in columns if column == 'text'},
            batched=True,
            num_proc=multiprocessing.cpu_count(),
            remove_columns=[column for column in columns if column != 'text']  # Keep the 'text' column for tokenization
        )
    return dataset

# Define the advanced loader function for tokenization
def advanced_tokenize_dataset(dataset, tokenize_function):
    for split in dataset.keys():
        dataset[split] = dataset[split].map(
            tokenize_function,
            batched=True,
            num_proc=multiprocessing.cpu_count(),
            remove_columns=['text']  # Remove the 'text' column after tokenization
        )
    return dataset

# Apply the advanced preprocessing function to the dataset
dataset = advanced_preprocess_dataset(dataset, advanced_preprocess_function)

# Apply the advanced tokenization function to the dataset
dataset = advanced_tokenize_dataset(dataset, advanced_tokenize_function)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map (num_proc=2):   0%|          | 0/153 [00:00<?, ? examples/s]

ValueError: Column name text not in the dataset. Current columns in the dataset: []

In [ ]:
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["train"][label_column]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=}")
print(f"{eval_preds[:100]=}")
print(f"{dataset['train'][label_column][:10]=}")

accuracy=0.0
eval_preds[:100]=['\xa0@HMRCcustomers No this is my', '\xa0"I am sorry for the inconvenience. Please', "\xa0I'm not sure if you're aware of", '\xa0@EE On Rosneath Arial having good', "\xa0(I'm not sure if it's a", "\xa0@mckelldogs I'm not sure what", '\xa0@Yelp can we get the exact', '\xa0@nationalgridus I have no water and', "\xa0I'm not sure if they're really that", '�� �� �� �� �� ', "\xa0@NortonSupport\nI'm not", '\xa0Verizon Support all of a sudden I can', '???????????????????? ', '�� �� �� �� �� ', "\xa0@IanJamesPoulter What's your", '\xa0@AWSSupport When will be able', '\xa0@NCIS_CBS\nThe first thing', '\xa0@msetchell\n@msetchell', '\xa0"I\'m not a duck!"\nI', '\xa0@NortonSupport @Norton', '\xa0United \xa0United \xa0United \xa0', "\xa0@ZARA_Care I've been waiting", '\xa0New Listing! Large 2 Family Home for', '\xa0@SouthwestAir I love you but when', '!!!\n\n:!!! Date :!!!', '\xa0@nvidiacc I own two g', '\xa0-\xa0-\xa0-\xa0-\xa0-', '\xa0Perdue Chicken - Perdue Chicken

In [ ]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)  # don't add bos token because we concatenate with inputs
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs